In [ ]:
import os
import time
import torch
import shutil
import random
import sklearn
import numpy as np
import torchvision
import torch.nn as nn
import seaborn as sns
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix

In [ ]:
# GPU Memory Reset
from numba import cuda
device = cuda.get_current_device()
device.reset()

# **Loading Data**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Remove folders from dataset
try:
  shutil.rmtree('EuroSAT_RGB')
  shutil.rmtree('__MACOSX')
  shutil.rmtree('imagens_treino')
  shutil.rmtree('imagens_teste')
except Exception as e:
  print(f'Folders not found')

Folders not found


In [ ]:
# Unzip dataset
!unzip /content/drive/MyDrive/Colab\ Notebooks/Cap9/EuroSAT_RGB.zip

In [ ]:
if not os.path.isdir('train_images'):
  os.mkdir('train_images')
if not os.path.isdir('test_images'):
  os.mkdir('test_images')

In [ ]:
source_images = 'EuroSAT_RGB'
destination_train = 'train_images'
destination_test = 'test_images'

# **Separating Images**

In [ ]:
image_class = 0
class_dict = {}

In [ ]:
files = os.listdir(source_images)
files.sort()

In [ ]:
# Iterate over all images
for file_path in files:
  # Check if the file is not a hidden file
  if file_path[0] != '.':
    # List all images in the current directory
    images = os.listdir(source_images + '/' + file_path)

    # Calculate sample size for training (80% of images)
    sample_size = int(len(images) * 0.8)

    # Initialize a list to store training image names
    train = []

    # Define the destination path for training images
    final_dest = destination_train + '/' + str(image_class)

    # Create the destination directory for training images
    os.mkdir(final_dest)

    # Copy a random sample of images to the training directory
    for file_name in random.sample(images, sample_size):
      shutil.copy2(os.path.join(source_images, file_path, file_name), final_dest)

      # Add the file name to the training list
      train.append(file_name)

    # Determine the test images by subtracting the training images from all images
    test_images = list(set(images) - set(train))

    # Define the destination path for test images
    final_dest = destination_test + '/' + str(image_class)

    # Create the destination directory for test images
    os.mkdir(final_dest)

    # Copy the test images to the test directory
    for test_image in test_images:
      shutil.copy2(os.path.join(source_images, file_path, test_image), final_dest)

    # Map the current class index to the file path in the class dictionary
    class_dict[image_class] = file_path

    # Increment the class index for the next iteration
    image_class += 1

# **PreProcessing and Creating DataLoader**

In [ ]:
# Data Transformation
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root = 'train_images', transform = transform)

dl_train = torch.utils.data.DataLoader(train_dataset, batch_size = 64, shuffle = True, num_workers = 2)

In [ ]:
test_dataset = torchvision.datasets.ImageFolder(root = 'test_images', transform = transform)

dl_test = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle = True, num_workers = 2)

# **Plotting Images**

In [ ]:
def imshow(img):
  img = img / 2 + 0.5

  npimg = img.numpy()

  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

In [ ]:
dataiter = iter(dl_train)
images, labels = next(dataiter)

In [ ]:
mapping = {0: 'AnnualCrop',
           1: 'Forest',
           2: 'HerbaceousVegetation',
           3: 'Highway',
           4: 'Industrial',
           5: 'Pasture',
           6: 'PermanentCrop',
           7: 'Residential',
           8: 'River',
           9: 'SeaLake'}

In [ ]:
imshow(torchvision.utils.make_grid(images[:8]))
print('Labels:', ' '.join('%d' % labels[j] for j in range(8)))

# **Creating Model**

In [ ]:
# Define a convolutional neural network (ConvNet) class
class ConvNet(nn.Module):
  # Initialization method for the class
  def __init__(self):
    # Call the initialization method of the parent class (nn.Module)
    super(ConvNet, self).__init__()

    # Define the first convolutional layer
    # This layer takes images with 3 channels (RGB), produces 64 feature maps,
    # uses filters of size 3x3, and a stride of 1
    self.conv1 = nn.Conv2d(3, 64, 3, 1)

    # Define the second convolutional layer
    # This layer takes the 64 feature maps from the previous layer and produces 128 new maps,
    # with filters of size 3x3 and a stride of 1
    self.conv2 = nn.Conv2d(64, 128, 3, 1)

    # Define the third convolutional layer
    # This layer takes the 128 feature maps from the previous layer and produces 256 new maps,
    # with filters of size 3x3 and a stride of 1
    self.conv3 = nn.Conv2d(128, 256, 3, 1)

    # Define a dropout layer with a rate of 25% for regularization
    # This helps prevent overfitting during training
    self.dropout1 = nn.Dropout(0.25)

    # Define a second dropout layer with a rate of 50%
    # Provides additional regularization in later layers
    self.dropout2 = nn.Dropout(0.5)

    # Define the first fully connected layer
    # This layer takes an input of size 215296 and outputs 2048 features
    self.fc1 = nn.Linear(215296, 2048)

    # Define the second fully connected layer
    # This layer takes the 2048 features from the previous layer and outputs 512 features
    self.fc2 = nn.Linear(2048, 512)

    # Define the third fully connected layer
    # This layer takes the 512 features from the previous layer and outputs 128 features
    self.fc3 = nn.Linear(512, 128)

    # Define the fourth fully connected layer
    # This layer takes the 128 features from the previous layer and outputs 10 features
    # Typically, this would correspond to the number of classes in a classification problem
    self.fc4 = nn.Linear(128, 10)

  def forward(self, x):
    # Apply the first convolutional layer
    x = self.conv1(x)

    # Apply ReLU activation function
    x = F.relu(x)

    # Apply the second convolutional layer
    x = self.conv2(x)

    # Apply ReLU activation function
    x = F.relu(x)

    # Apply the third convolutional layer
    x = self.conv3(x)

    # Apply ReLU activation function
    x = F.relu(x)

    # Apply max pooling with a kernel size of 2
    x = F.max_pool2d(x, 2)

    # Apply the first dropout layer
    x = self.dropout1(x)

    # Flatten the tensor while keeping the batch size (dimension 0) intact
    x = torch.flatten(x, 1)

    # Apply the first fully connected layer
    x = self.fc1(x)

    # Apply ReLU activation function
    x = F.relu(x)

    # Apply the second dropout layer
    x = self.dropout2(x)

    # Apply the second fully connected layer
    x = self.fc2(x)

    # Apply ReLU activation function
    x = F.relu(x)

    # Apply the third fully connected layer
    x = self.fc3(x)

    # Apply ReLU activation function
    x = F.relu(x)

    # Apply the fourth fully connected layer
    x = self.fc4(x)

    # Apply log softmax activation function for the output
    return F.log_softmax(x, dim=1)


In [ ]:
model = ConvNet()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
model.to(device)

# **Selecting Loss Function**

In [ ]:
loss_function = nn.CrossEntropyLoss()

# **Selecting Optmizer Function**

In [ ]:
optimizer = optim.Adam(model.parameters())

# **Training the Model**

In [ ]:
epochs = 30

In [ ]:
testiter = next(dl_test)

In [ ]:
%%time

print('Training Started')

# Loop through each epoch
for epoch in range(epochs):
  # Initialize the running loss for the epoch
  running_loss = 0.0

  i = 0

  # Loop through the training data
  for data in (pbar := tqdm(dl_train)):
    # Update the progress bar description with the current epoch
    pbar.set_description(f'\nEpoch: {epoch}')

    inputs, labels = data

    # Move the inputs and labels to the specified device (e.g., GPU)
    inputs, labels = inputs.to(device), labels.to(device)

    # Zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass: compute the model output for the inputs
    outputs = model(inputs)

    # Compute the loss
    loss = loss_function(outputs, labels)

    # Backward pass: compute the gradients
    loss.backward()

    # Update the model parameters
    optimizer.step()

    # Accumulate the loss
    running_loss += loss.item()

    total_correct = 0
    total_samples = 0

    # Perform validation/testing every 100 iterations
    if i % 100 == 0:
      # Disable gradient calculation for validation/testing
      with torch.no_grad():
        # Get a batch of test images and labels
        test_images, test_labels = next(testiter)
        test_images, test_labels = test_images.to(device), test_labels.to(device)

        test_outputs = model(test_images)

        # Get the predicted class labels
        _, predicted = torch.max(test_outputs, 1)

    i += 1

    print(f'Epoch: {epoch}, Loss: {running_loss / (i)}')

print('Training Completed')
